In [ ]:
import pandas as pd

games = pd.read_csv('../datasets/potential_final.csv')
games

In [ ]:

games = pd.read_csv('../datasets/potential_final.csv')

games.drop(['Unnamed: 0'], axis=1, inplace=True)
games = games.fillna('')
games['release_date'] = pd.to_datetime(games['release_date']).dt.strftime("%Y-%m-%dT%H:%M:%SZ")

games.head()

In [ ]:
from ast import literal_eval

# In some entries the user review is marked as "tbd", this isn't compatible with Solr's FloatPointField type so it should be stored as "NaN" instead
def get_user_review(user_review):
    if user_review == "tbd":
        return "NaN"
    return user_review

def get_platform_reviews_json(game):
    platforms = list(games[games.name == game]["platform"])
    json = []

    for platform in platforms:
        reviews = (games[(games.name == game) & (games.platform == platform)]).iloc[0].MetacriticReviews
        for review in eval(reviews):
            json.append({
                "platform": (games[(games.name == game) & (games.platform == platform)]).iloc[0].platform,
                "release_date": (games[(games.name == game) & (games.platform == platform)]).iloc[0].release_date,
                "meta_score": str((games[(games.name == game) & (games.platform == platform)]).iloc[0].meta_score),
                "user_review": get_user_review((games[(games.name == game) & (games.platform == platform)]).iloc[0].user_review),
                "reviewer": review[0],
                "review": review[1]
            })
    return json

def get_game_json(game):
    json = {
        "name": game,
        "summary": (games[games.name == game]).iloc[0].summary,
        "wikipedia": (games[games.name == game]).iloc[0].Wikipedia,
        "genre": (games[games.name == game]).iloc[0].Genre,
        "playing_modes": literal_eval((games[games.name == game]).iloc[0].Modes) if (games[games.name == game]).iloc[0].Modes else [],
        "content_type": "parentDocument",
        "reviews": get_platform_reviews_json(game)
    }
    return json

In [ ]:
from json import dumps

json = []
counter = 0
for game in games['name'].unique():
    print(counter)
    counter = counter +1
    json.append(get_game_json(game))

json = dumps(json, indent=4)
with open("../search-engine/games_collection.json", "w") as outfile:
    outfile.write(json)